In [4]:
import time
import numpy as np
import pandas as pd
import datetime as dt
from pylab import mpl, plt

import websocket, json

In [5]:
import seaborn as sns
# Apply seaborn style
sns.set()

mpl.rcParams['font.family'] = 'serif'
%matplotlib inline

In [6]:
import os
import numpy as np
import pandas as pd
import datetime
import time
from datetime import date
import pandas_ta as ta
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from yahoo_fin import stock_info as si
import datetime
import yfinance as yf
from random import randint
from time import sleep

In [7]:
def load_historic_data(symbol, start_date_str, today_date_str, period, interval, prepost):
    try:
        df = yf.download(symbol, start=start_date_str, end=today_date_str, period=period, interval=interval, prepost=prepost)
        
        # print(df.head(5))

        #  Add symbol
        df["Symbol"] = symbol
        
        return df
    except:
        print('Error loading stock data for ' + symbol)
        return None

In [8]:
# valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
# fetch data by interval (including intraday if period < 60 days)
        # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
period = '1d'
interval = '1m'
prepost = True

today = datetime.date.today()
today_date_str = today.strftime("%Y-%m-%d")

#  NOTE: 7 days is the max allowed
days = datetime.timedelta(7)
start_date = today - days
start_date_str = datetime.datetime.strftime(start_date, "%Y-%m-%d")

#  Coins to download
# symbols = ['BTC-USD','ETH-USD','BCH-USD','LTC-USD','BNB-USD','BAT-USD',
#            'XLM-USD','DOGE-USD','DOGE-USD','COMP-USD','ALGO-USD','OMG-USD']
symbols = ['BTC-USD', 'ETH-USD']

In [9]:
#  Fetch data for coin symbols
for symbol in symbols:
    print(f"Loading data for {symbol}")
 
    # Load data
    df = load_historic_data(symbol, start_date_str, today_date_str, period, interval, prepost)
    
    # Reset index to move datetime index to a column
    df.reset_index(inplace=True)
    
    # print(df.head(5))

    # Generate dynamic filename
    file_name = f"{today_date_str}_{symbol}_{interval}.csv"
    # Save df to CSV in the current working directory
    df.to_csv(file_name, index=False)
    print(f"Data saved to {file_name}\n")

    #  Avoid DOS issues
    sleep(randint(0,3))

Loading data for BTC-USD


[*********************100%%**********************]  1 of 1 completed


Data saved to 2024-07-05_BTC-USD_1m.csv

Loading data for ETH-USD


[*********************100%%**********************]  1 of 1 completed


Data saved to 2024-07-05_ETH-USD_1m.csv



In [10]:
def plot_graph(df):
    # Calculate Mid price
    df['Mid'] = (df['High'] + df['Low']) / 2
    
    # Calculate Simple Moving Average (SMA)
    df['SMA'] = df['Mid'].rolling(window=1000).mean()

    fig = make_subplots(rows=1, cols=1)
    # Plot close price
    # fig.add_trace(go.Line(x = df.index, y = df['Close'], line=dict(color="blue", width=1), name="Close"), row = 1, col = 1)
    
    # Plot Mid price
    fig.add_trace(go.Scatter(x=df.index, y=df['Mid'], line=dict(color="black", width=1), name="Mid"), row=1, col=1)
    
    # Plot SMA
    fig.add_trace(go.Scatter(x=df.index, y=df['SMA'], line=dict(color="orange", width=1), name="SMA"), row=1, col=1)

    fig.update_layout(
        title={'text':'7 days, 1m interval', 'x':0.5},
        autosize=False,
        width=800,
        height=400)
    fig.update_yaxes(range=[0,1000000000],secondary_y=True)
    fig.update_yaxes(visible=False, secondary_y=True)  #hide range slider
    
    fig.show()

In [11]:
df = load_historic_data('BTC-USD', start_date_str, today_date_str, period, interval, prepost)
plot_graph(df)

[*********************100%%**********************]  1 of 1 completed


In [12]:
print(df.head(5))

                                   Open          High           Low  \
Datetime                                                              
2024-06-28 00:01:00+00:00  61613.207031  61613.207031  61613.207031   
2024-06-28 00:02:00+00:00  61575.140625  61575.140625  61575.140625   
2024-06-28 00:04:00+00:00  61596.730469  61596.730469  61596.730469   
2024-06-28 00:05:00+00:00  61577.593750  61577.593750  61577.593750   
2024-06-28 00:06:00+00:00  61575.207031  61575.207031  61575.207031   

                                  Close     Adj Close   Volume   Symbol  \
Datetime                                                                  
2024-06-28 00:01:00+00:00  61613.207031  61613.207031        0  BTC-USD   
2024-06-28 00:02:00+00:00  61575.140625  61575.140625        0  BTC-USD   
2024-06-28 00:04:00+00:00  61596.730469  61596.730469  1658880  BTC-USD   
2024-06-28 00:05:00+00:00  61577.593750  61577.593750        0  BTC-USD   
2024-06-28 00:06:00+00:00  61575.207031  61575.20703